In [1]:
import json
import random
from typing import Any, List
import yaml

from jinja2 import Template, StrictUndefined

from openai import AsyncOpenAI
from pydantic import BaseModel, Extra, Field

from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel, OpenAIResponsesModelSettings
from pydantic_ai.providers.openai import OpenAIProvider

from src.models import (
    CharacterSpecification,
    CharacterState,
    Character,
    EmotionalState,
    SceneBackground,
    create_dynamic_enum
)
from src.action_predictor import (
    CharcterActionPredictor
)
from src.character_store import CharacterStore

from config import settings

In [2]:
# Initialize OpenAI provider
print(settings.llm_model)

# Pydantic Model
provider = OpenAIProvider(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)
model = OpenAIModel(
    model_name=settings.llm_model,
    provider=provider
)

# OpenAI Client
client = AsyncOpenAI(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)

gpt-4.1-nano


# Load Scene & Characters

In [3]:
# Sample Scene
# utilized trope: "Defrosting Ice Queen", "SavvyGuyEnergeticGirl"

scene_outline="Inside the bustling grandeur of the metropolitan fashion gala, Zephyr Orion, a 28-year-old jocular astronaut with a penchant for playful storytelling, encounters Vivienne LaRoux. Vivienne, also 28, exudes sophistication and an assertive demeanor as a renowned style influencer. Initially, she greets Zephyr's lighthearted banter with icy indifference, her mean streak surfacing sporadically. However, Zephyr's infectious humor gradually softens her edges, revealing a subtly receptive side. Their verbal dance, rich with lively exchanges, challenges both to reconsider their outlooks, Zephyr embracing Vivienne's world of high fashion while she discovers a brighter perspective in his social magnetism."

scene_background = {
    "location": "The Metropolitan Fashion Gala",
    "setting": "An opulent event hall adorned with sparkling chandeliers and cutting-edge fashion displays.",
    "explanation": "In the midst of a vibrant and luxurious fashion gala, Zephyr Orion and Vivienne LaRoux engage in a compelling social interaction. Zephyr, with his distinctive brand of humor and affable nature, engages Vivienne in a conversation that is both disarming and endearing. Their exchange, a medley of banter and keen observations, reflects not only their mutual intrigue but also hints at a budding dynamic that bridges their contrasting worlds of space and style. As the evening unfolds, both characters experience subtle shifts in their perspectives, opening their minds to each other's unique lives, and suggesting the start of a meaningful connection."
}
scene_background = SceneBackground(**scene_background)

In [4]:
character_uids = [
    "35f0c56f-263d-42df-846c-e1833d8ca0ab",
    "00d66087-9b3b-46da-bd74-bf45cbe81d3c"
]

character_store = CharacterStore(
    character_dir="assets/characters",
    uids=character_uids
)

with open("assets/sims_interactions.json", "r") as f:
    all_actions = json.load(f)
n = 10
actions = {k: random.sample(v, min(len(v), n)) for k,v in all_actions.items()}
print(actions.keys())

dict_keys(['Friendly', 'Ask', 'Romantic', 'Mean', 'Neutral'])


In [5]:
# get scene characters
scene_characters = {
    uid: character_store.get(uid).spec
    for uid in character_uids
}

In [6]:
scene_characters

{'35f0c56f-263d-42df-846c-e1833d8ca0ab': CharacterSpecification(name='Zephyr Orion', gender='male', age=28, dialogue_tone='playful, jovial, and engaging, with a witty humor and warmth that makes everyone feel at ease. Known for storytelling with captivating tales of space adventures.', career='Astronaut', personality_traits=[PersonalityTrait(trait='Goofball', description='Enjoys joking and making others laugh, bringing a playful spirit to social situations.'), PersonalityTrait(trait='Materialistic', description='Loves acquiring new possessions and often leans towards bragging about them.'), PersonalityTrait(trait='Outgoing', description='Flourishes in social situations and enjoys being around people.'), PersonalityTrait(trait='Gloomy', description='Grows sad when left alone for too long.'), PersonalityTrait(trait='Ambitious', description='Continuously strives to reach new milestones in his career.')], hobbies=['Fitness', 'Cooking', 'Painting'], living_conditions=['Resides in a modern c

# Initialize Agent

In [7]:
# Initialize Action Predictor
with open('prompts/action_predictor.yaml', 'r') as file:
    action_predictor_prompt = yaml.load(file, Loader=yaml.FullLoader)

predictor = CharcterActionPredictor(
    client=client,
    prompt=action_predictor_prompt,
    actions=actions,
    character_store=character_store,
    model="gpt-4.1-nano"
)

In [8]:
class Action(BaseModel):
    character_uid: str
    action: str
    targets: List[str]
    
    class Config:
        extra = "forbid"
        use_enum_values = True
    
with open('prompts/planning_agent.yaml', 'r') as file:
    agent_prompt = yaml.load(file, Loader=yaml.FullLoader)
    

    
agent = Agent(
    model=model,
    name="planning",
    output_type=Action,
    system_prompt = agent_prompt['system']
)

In [9]:
# register tool

@agent.tool
async def predict_action(
    ctx: RunContext[None],
    scene_context: str,
    character_uid: str,
    related_character_uids: List[str]
) -> Action:
    
    predicted_action = await predictor.forward(
        scene_context=scene_context,
        uid="35f0c56f-263d-42df-846c-e1833d8ca0ab",
        related_uids=related_character_uids
    )
    action = Action(
        character_uid=character_uid,
        action=predicted_action['action'],
        targets=predicted_action['targets']
    )
    return action

In [10]:
# render user message
user_template  = Template(
    agent_prompt['user'],
    undefined=StrictUndefined
)

contents = {
    "scene_outline": scene_outline,
    "scene_background": scene_background,
    "characters": scene_characters,
    "history": []
}
user_message = user_template.render(**contents)

In [11]:
print(user_message)

Only predict one action step
[Scene Description]
* Outline: Inside the bustling grandeur of the metropolitan fashion gala, Zephyr Orion, a 28-year-old jocular astronaut with a penchant for playful storytelling, encounters Vivienne LaRoux. Vivienne, also 28, exudes sophistication and an assertive demeanor as a renowned style influencer. Initially, she greets Zephyr's lighthearted banter with icy indifference, her mean streak surfacing sporadically. However, Zephyr's infectious humor gradually softens her edges, revealing a subtly receptive side. Their verbal dance, rich with lively exchanges, challenges both to reconsider their outlooks, Zephyr embracing Vivienne's world of high fashion while she discovers a brighter perspective in his social magnetism.
* Background:
  * location: The Metropolitan Fashion Gala
  * setting: An opulent event hall adorned with sparkling chandeliers and cutting-edge fashion displays.
  * explanation: In the midst of a vibrant and luxurious fashion gala, Zep

In [12]:
result = await agent.run(user_message)

Information:
[Character]
description:
{"name":"Zephyr Orion","gender":"male","age":28,"dialogue_tone":"playful, jovial, and engaging, with a witty humor and warmth that makes everyone feel at ease. Known for storytelling with captivating tales of space adventures.","career":"Astronaut","personality_traits":[{"trait":"Goofball","description":"Enjoys joking and making others laugh, bringing a playful spirit to social situations."},{"trait":"Materialistic","description":"Loves acquiring new possessions and often leans towards bragging about them."},{"trait":"Outgoing","description":"Flourishes in social situations and enjoys being around people."},{"trait":"Gloomy","description":"Grows sad when left alone for too long."},{"trait":"Ambitious","description":"Continuously strives to reach new milestones in his career."}],"hobbies":["Fitness","Cooking","Painting"],"living_conditions":["Resides in a modern city apartment filled with space memorabilia and art supplies.","Leads a healthy lifesty

In [13]:
result

AgentRunResult(output=Action(character_uid='35f0c56f-263d-42df-846c-e1833d8ca0ab', action="Zephyr chuckles softly and gestures with a playful wink, leaning in slightly as he shares a humorous space anecdote. 'You know, in the orbit of my last mission, even the stars were jealous of my shine.'", targets=['00d66087-9b3b-46da-bd74-bf45cbe81d3c']))

In [14]:
print(result.output.model_dump_json(indent=2))

{
  "character_uid": "35f0c56f-263d-42df-846c-e1833d8ca0ab",
  "action": "Zephyr chuckles softly and gestures with a playful wink, leaning in slightly as he shares a humorous space anecdote. 'You know, in the orbit of my last mission, even the stars were jealous of my shine.'",
  "targets": [
    "00d66087-9b3b-46da-bd74-bf45cbe81d3c"
  ]
}
